# Compute Lasso Paths

## Overview
The goal is to detect 6-mers that affect mRNA stability after taking into account codon optimality (PLS components). We know that MiR-430 k-mers are the strongest regulators, hence ariel suggested to implement this analysis only for the genes without MiR-430 sites, but I think is not the best approach. I will run 2 times the analysis by deliting all MiR-430 sites and with MiR-430 site. I also run this analysis for the two RNA protocols poly-a & ribo0.


In [1]:
import os
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import itertools
from multiprocessing import Pool

# analysis code
from lassoloaddata import loadlassodata
from lassopath import lasssopath


warnings.filterwarnings('ignore')
%matplotlib inline

We have to different data set ribo and polyA

In [2]:
def runanalysis(rna_protocol='wt_ribo', keepmir=True):
    """
    runs the analysis to get the lasso path in the given data
    """
    predictors, response = loadlassodata(rna_protocol)
    mirstatus = "all_genes" if keepmir else "no_mir_genes"
    # remove MiR-430 genes
    if not keepmir:
        no_mir_genes = predictors['GCACTT'] == 0 # 0 6-mer MiR-430 sites
        predictors = predictors[no_mir_genes]
        response = response[no_mir_genes]
    # run the lasso with 6 folds, the predictors are scaled
    print('runing analysis for {} genes'.format(predictors.shape[0]))
    paths = lasssopath(predictors, response)
    paths['sample_condition'] = rna_protocol
    paths['which_genes'] = mirstatus
    return paths

In [3]:
# RUN THE ANALYSIS
results = list()
fargs = [(cond, mircase) for cond, mircase in itertools.product(
    ("wt_ribo", "wt_polya"), (True, False))]

In [4]:
def runparallel(argstupple):
    # this function is to run the analysis in parallel
    a, b = argstupple
    return runanalysis(a, b)
cores = 4
with Pool(processes=4) as pool:
    results = pool.map(runparallel, fargs)

runing analysis for 4729 genes
runing analysis for 4729 genes
pass ..
pass ..
runing analysis for 3349 genes
runing analysis for 3349 genes
runing fold 1 of 6
runing fold 1 of 6
runing fold 1 of 6
runing fold 1 of 6
runing fold 2 of 6
runing fold 2 of 6
runing fold 2 of 6
runing fold 3 of 6
runing fold 3 of 6
runing fold 2 of 6
runing fold 3 of 6
runing fold 4 of 6
runing fold 4 of 6
runing fold 3 of 6
runing fold 4 of 6
runing fold 5 of 6
runing fold 4 of 6
runing fold 5 of 6
runing fold 5 of 6
runing fold 5 of 6
runing fold 6 of 6
runing fold 6 of 6
runing fold 6 of 6
runing fold 6 of 6


In [8]:
# save results
pd.concat(results).to_csv('results_data/lasso_paths.csv')